<a href="https://colab.research.google.com/github/lariyumi/CodigoSeminarioPLN/blob/main/SeminarioPLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introdução**

Nesse artigo, é mostrado como criar, usando agentes, uma pipeline de ciência de dados ponta a ponta, que realizam tarefas como coleta de dados, pré-processamento, treinamento e avaliação, de forma que no final, teremos uma ferramenta similar com o GitHub Copilot (assistente IA que ajuda no desenvolvimento do seu código, sugerindo as próximas linhas ou alguma função específica, de acordo com o contexto), mas para um projeto inteiro.

Apesar de ter várias etapas para o desenvolvimento de um projeto, esse artigo busca automatizar apenas 3: coleta de dados, pré-processamento e treinamento.

Teremos então três agentes: um de coleta de dados, que vai ser responsável por achar datasets no Kaggle -uma comunidade para aprendizado em ciência de dados que possui também datasets para treinamento em Data Science e Machine Learning-, um que realiza tarefas de pré-processamento de dados em um processo sequencial -carregando o conjunto de dados e analisando duplicatas, valores ausentes etc-, e por fim, um que realiza o treinamento de modelos -criando uma ferramenta personalizada para treinar um modelo de aprendizado de máquina-.

Link para o artigo: https://generativeai.pub/creating-an-end-to-end-data-science-project-with-crewai-agents-f98d02b4e203

# **Código**

Primeiramente, instalamos os pacotes que iremos usar em nosso código

In [ ]:
!pip install 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.7/462.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain-cohere

Começamos criando uma ferramenta customizada, que é um componente que realizará uma tarefa específica, a partir do BaseTool do CrewAI para baixar datasets do Kaggle

  


In [ ]:
from crewai_tools import BaseTool
from kaggle.api.kaggle_api_extended import KaggleApi

class KaggleDatasetDownloader(BaseTool):
  name: str = "Kaggle Dataset Downloader"
  description: str = "Baixa datasets do Kaggle usando a URL dada."

  def _run(self, url: str) -> str:
    try:
      #Realiza a autenticação na API
      api = KaggleApi()
      api.authenticate()
      #Extrai a informação do dataset passado pela URL
      parts = url.split('/')
      owner = parts[-2]
      dataset_name = parts[-1]

      #Baixa o dataset para um diretório especificado
      api.dataset_download_files(f"{owner}/{dataset_name}", path='./raw_data', unzip=True)
      return f"Dataset Baixado com sucesso: {owner}/{dataset_name} para o diretório ./raw_data "
    except Exception as e:
      if '403' in str(e):
        #Tratamento de erro para o caso de não conseguir acessar a página
        return "Erro 403: Forbidden. Cheque suas credenciais do Kaggle API e as permissões do dataset."
      else:
        #Tratamento de erro para o caso da página ser acessada, mas por algum motivo ocorrer um erro ao tentar baixar os dados
        return f"Erro baixando o dataset: {str(e)}"

Agora, nós definimos as senhas de todas as APIs utilizadas e criamos o agente que realizará a coleta dos dados na Kaggle API

In [ ]:
from crewai import Agent
import os
from langchain_cohere import ChatCohere
#from langchain_groq import ChatGroq
from crewai_tools import SerperDevTool
from google.colab import userdata

#Definimos as chaves das APIs que serão utilizadas
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')
#os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

#Definimos o LLM que será utilizado e também definimos um valor para a temperatura, que determinaria o quanto o LLM será criativo ao nos dar uma resposta
llm = ChatCohere(model="command-r-plus-08-2024", temperature=0)
#llm = ChatGroq(model='llama3-8b-8192',temperature=0)

#Carregamos o Serper
serper_search_tool = SerperDevTool()

#Carregamos a ferramenta que criamos
kaggle_tool = KaggleDatasetDownloader()

#Criamos o agente de coleta de dados (agentes utilizam chains para interagir com o ambiente e tomar decisões)
data_collection_agent = Agent(
    role='Data Acquisition Specialist',
    goal='Find and download appropriate datasets on a given topic',
    backstory='Expert in acquiring datasets from various sources, specializing in climate data.',
    tools=[serper_search_tool, kaggle_tool],
    llm=llm,
    verbose=True#permite que visualizemos seus inputs e outputs
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Criamos outra ferramenta para realizar o pré-processamento dos dados. Ela irá lidar com dados duplicados, dados que estão faltando e codificar variáveis categóricas, que são variáveis expressas por um número finito de rótulos, que podem ser numéricos ou não numéricos.



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

class DataPreprocessor(BaseTool):
  name: str = "Data Preprocessor"
  description: str = "Realiza o pré-processamento dos dados lidando com valores que estão faltando, removendo dados duplicados e codificando variáveis categóricas"

  def _run(self, file_path: str) -> str:
    #carregamos os dados
    df = pd.read_csv(file_path)

    #Pegamos as informações iniciais
    initial_shape = df.shape
    initial_missing = df.isnull().sum().sum()

    #Lidando com os valores faltando
    df = df.dropna() #remove do dataframe os dados com valores faltando

    #Removendo dados duplicados
    df = df.drop_duplicates()

    #Identificando as colunas categóricas
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

    #Convertendo variáveis categóricas em variáveis numéricas
    label_encoder = LabelEncoder()
    for col in categorical_columns:
      df[col] = label_encoder.fit_transform(df[col])

    #Pegamos as informações finais
    final_shape = df.shape
    final_missing = df.isnull().sum().sum()

    #Salvamos os dados processados
    processed_file_path = os.path.join('dados_processados.csv')
    df.to_csv(processed_file_path, index=False)

    return f"""
    Pré-processamento de dados completo:
    - Forma Inicial: {initial_shape}
    - Valores Faltando inicialmente: {initial_missing}
    - Forma final: {final_shape}
    - Valores faltando no final: {final_missing}
    - Variáveis categóricas criadas: {categorical_columns}
    - Dados duplicados removidos
    - Dados processados salvos em: {processed_file_path}
    """

Agora nós criamos o agente que realizará essa tarefa de pré-processamento dos dados

In [ ]:
from crewai_tools import DirectoryReadTool

docs_tool_a = DirectoryReadTool(directory='raw_data')

data_processing_tool = DataPreprocessor()

#definimos o agente de pré-processamento de dados
data_preprocessing_agent = Agent(
    role="Data Preprocessing Specialist",
    goal="Load, clean, and perform initial transformations on datasets",
    backstory="Expert in data cleaning and preprocessing using pandas, numpy, and sklearn libraries",
    llm=llm,
    tools=[docs_tool_a, data_processing_tool],
    verbose=True
)

Para testar os agentes, criaremos tasks para eles executar

In [ ]:
from crewai import Crew, Task
from textwrap import dedent

#Criamos a task para o agente de coleta de dados
data_collection_task = Task(
    description="""
    Search for three appropriate datasets on the topic of {topic} and download one using the Kaggle Dataset Downloader.
    You can search for datasets using refined queries. Note that the Kaggle Dataset Downloader only requires one input, i.e., the URL.
    """,
    expected_output = 'Provide the full description of the downloaded dataset.',
    agent = data_collection_agent
)

#Criamos a task para o agente de pré-processamento de dados
data_preprocessing_task = Task(
    description="""
    Load the file from {file_path}, handle missing values, remove duplicates, and convert categorical variables to numerical values to make the dataset model-ready.
    """,
    expected_output = 'Processed dataset saved successfully',
    agent = data_preprocessing_agent
)

Usamos então o CrewAI para realizar as tasks que acabamos de definir para os agentes através da criação de um crew

In [ ]:
#Criamos um Crew, em que definimos um grupo de agentes e as tarefas. Cada crew define qual agente será utilizado para cada task e o fluxo de trabalho geral
crew = Crew(
    agents=[data_collection_agent, data_preprocessing_agent],
    tasks=[data_collection_task, data_preprocessing_task],
    verbose=True
)
#realizamos a operação de pesquisar, coletar e armazenar os dados sobre o tópico desejado, além de realizar o pré-processamento dos dados que acabamos de coletar
result = crew.kickoff(inputs={'topic': 'housing', 'file_path': 'housing'})

 [2024-09-05 13:33:24][DEBUG]: == Working Agent: Data Acquisition Specialist
 [2024-09-05 13:33:24][INFO]: == Starting Task: 
    Search for three appropriate datasets on the topic of housing and download one using the Kaggle Dataset Downloader.
    You can search for datasets using refined queries. Note that the Kaggle Dataset Downloader only requires one input, i.e., the URL.
    


> Entering new CrewAgentExecutor chain...
Thought: To find appropriate datasets on housing, I will search for relevant keywords on Kaggle. I can then download one of the datasets using the provided tool.

Action: Search the internet
Action Input: {"search_query": "housing datasets kaggle"}

Observation 


Search results: Title: Housing Prices Dataset | Kaggle
Link: https://www.kaggle.com/datasets/yasserh/housing-prices-dataset
Snippet: Housing Prices Prediction - Regression Problem.
---
Title: Housing Dataset - Kaggle
Link: https://www.kaggle.com/datasets/ashydv/housing-dataset
Snippet: Kaggle is the worl

Vamos agora começar a fazer o agente que vai realizar os treinamentos de modelos. Primeiramente faremos a ferramenta que será utilizada em tal treinamento

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import pickle

class TrainingModelTool(BaseTool):
  name: str = "Random Forest Model Training"
  description: str = "Trains a Random Forest model for house price prediction and saves it as a pickle file"

  def _run(self, file_path: str, target_variable: str)-> str:
    #Carrega e prepara os dados
    data = pd.read_csv(file_path)
    X = data.drop(target_variable, axis=1) #valores sem ser a coluna que passamos por parâmetro
    y = data[target_variable] #valores que são da coluna que passamos por parâmetro

    #Separa os dados em matrizes de treino aleatórias
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #Dimensionamos nossas features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.values)
    X_test_scaled = scaler.transform(X_test.values)

    #Treinamos o Random Forest Model (método de aprendizado)
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train_scaled, y_train)

    #Realiza previsões
    y_pred = model.predict(X_test_scaled)

    #Avaliamos os erros do modelo
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    #Importância das features
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    #Cria uma pasta chamada 'modelos_salvos' se não existir
    os.makedirs('modelos_salvos', exist_ok=True)

    #Cria uma pasta chamada 'dados_treino_teste' se não existir
    os.makedirs('dados_treino_teste', exist_ok=True)

    #salva o modelo na pasta 'modelos_salvos' que criamos
    model_filename = os.path.join('modelos_salvos', 'random_forest_model.pkl')
    with open(model_filename, 'wb') as file:
      pickle.dump(model, file)

    #Salva o scaler na pasta 'modelos_salvos'
    scaler_filename = os.path.join('modelos_salvos', 'scaler.pkl')
    with open(scaler_filename, 'wb') as file:
      pickle.dump(scaler, file)

    #Salvamos os dados de treino e teste na pasta 'dados_treino_teste'
    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    train_filename = os.path.join('dados_treino_teste', 'dados_treino.csv')
    test_filename = os.path.join('dados_treino_teste', 'dados_teste.csv')

    train_data.to_csv(train_filename, index=False)
    test_data.to_csv(test_filename, index=False)

    report = f"Relatório de treino com o Random Forest Model:\n\n"
    report += f"Erro médio da raíz quadrada: ${rmse:.2f}\n"
    report += f"Pontuação do coeficiente de regressão: {r2:.4f}\n\n"
    report += "Top 5 características mais importantes"
    for _, row in feature_importance.head().iterrows():
      report += f"- {row['feature']}: {row['importance']:.4f}\n"
    report += f"\nModelo salvo como: {os.path.abspath(model_filename)}\n"
    report += f"Scaler salvo como: {os.path.abspath(scaler_filename)}\n"
    report += f"Dados de treino salvos como: {os.path.abspath(train_filename)}\n"
    report += f"Dados de teste salvos como: {os.path.abspath(test_filename)}\n"

    return report

Criamos agora o agente para realizar o treinamento de um modelo.

In [ ]:
#Usamos o DirectoryReadTool para acessar os dados dentro da pasta 'dados_processados'
#docs_tool_b = DirectoryReadTool(directory='dados_processados')

model_training_tool = TrainingModelTool()

#Criamos o agente responsável por treinar o modelo
model_training_agent = Agent(
    role="Random Forest Model Trainer",
    goal="Train an Random Forest model for the dataset",
    backstory="You are an expert in machine learning, specializing in Random Forest for regression/classification tasks.",
    tools=[model_training_tool],
    llm=llm,
    verbose=True
)

Criamos então a Task e executamos o agente de treinamento. A variável que iremos usar como base varia para cada tabela que o agente acaba por pegar na coleta de dados. Se for treinado com sucesso, veremos que os arquivos que definimos na tool serão criados, juntamente com um arquivo que conterá o relatório de treinamento.

In [ ]:
target_variable = input() #Coloque a variável que você deseja prever com o modelo treinado
target_directory = os.path.join('dados_processados.csv')

#Criamos a Task para o treinamento do modelo
model_training_task = Task(
    description=dedent(f"""
    Load the processed data from the directory 'dados_processados'. Train a Random Forest model and save the trained model.
    Note that TrainingModelTool._run() has two obrigatory positional arguments which are file_path, which has the value {target_directory}, and the {target_variable}.
    Remember passing these values when running the tool. (Ex: TrainingModelTool._run({target_directory}, {target_variable})).
    """),
    expected_output="Model trained successfully",
    output_file='reports/training_report.txt',
    agent=model_training_agent
)

crew = Crew(
    agents=[model_training_agent],
    tasks=[model_training_task],
    verbose=True
)

#Executamos o crew com o agente que vai treinar o modelo e a task que definimos para que ele execute.
result = crew.kickoff()

 [2024-09-05 01:33:21][DEBUG]: == Working Agent: Random Forest Model Trainer
 [2024-09-05 01:33:21][INFO]: == Starting Task: 
Load the processed data from the directory 'dados_processados'. Train a Random Forest model and save the trained model.
Note that TrainingModelTool._run() has two obrigatory positional arguments which are file_path, which has the value dados_processados.csv, and the price.
Remember passing these values when running the tool. (Ex: TrainingModelTool._run(dados_processados.csv, price)).



> Entering new CrewAgentExecutor chain...
Thought: I will start by loading the data from the specified directory and then train a Random Forest model for house price prediction using the provided tool.

Action: Random Forest Model Training
Action Input: {
    "file_path": "dados_processados.csv",
    "target_variable": "price"
}

Observation 

Relatório de treino com o Random Forest Model:

Erro médio da raíz quadrada: $1400765.84
Pontuação do coeficiente de regressão: 0.6118

To